In [ ]:
import pandas as pd
from models import base_res_net, small_res_net, efficient_net, pretrained_mobilenet, big_model
import tensorflow as tf
import tensorflow.compat.v1 as tfc
from sklearn.utils.class_weight import compute_class_weight

from src import InputPipeline

%load_ext autoreload
%autoreload 2

## Check GPU for tf

In [ ]:
# Some GPU setup
# for documentation about using gpus refer to: https://www.tensorflow.org/install/pip#windows-wsl2

tf.keras.backend.clear_session()

device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
try:  # prevent a previous session from being alive
  sess.close() 
except:
  pass

tfc.enable_eager_execution()
gpu_options= tfc.GPUOptions(per_process_gpu_memory_fraction = 0.8)
sess = tfc.InteractiveSession(config=tfc.ConfigProto(gpu_options=gpu_options))

### Optional Stuff
- Here we compute classweights since the data is very imbalanced

In [ ]:
train_df = pd.read_csv("../data/train_images_stratified.csv")

In [ ]:
class_weights = compute_class_weight(class_weight = 'balanced', classes = train_df["label"].unique(), y=train_df["label"])
class_weights = dict(enumerate(class_weights))

## Configure Parameters

In [ ]:
INPUT_SHAPE = (224,224,3)

CONF = {
  "learning_rate": 0.00001,
  "batch_size": 16,
  "epochs": 35,
  "loss_function": "sparse_categorical_crossentropy",
  "metric": "categorical_accuracy",
}

## Make Input Pipelines

In [ ]:
# Input pipeline for subspecies
sub_species_input_pipeline = InputPipeline(splits=(0.85,0.0,0.15), channels=3, batch_size=CONF["batch_size"], size=INPUT_SHAPE[:2])
# This has a stratified split
sub_species_input_pipeline.make_stratified_train_dataset(
  train_ds_path= "../data/train_ds_images_stratified.csv",
  val_ds_path="../data/val_ds_images_stratified.csv",
)

# Input pipeline for species
species_input_pipeline = InputPipeline(splits=(0.85,0.0,0.15), channels=3, batch_size=CONF["batch_size"], size=INPUT_SHAPE[:2])
species_input_pipeline.make_train_datasets(directory="../data/train_images/species_classify")  # This doesnt have a stratified split

In [ ]:
# Cache and prefetch datasets for faster access
sub_species_cached_train, sub_species_cached_val = sub_species_input_pipeline.get_cached_train_datasets()
species_cached_train, species_cached_val = species_input_pipeline.get_cached_train_datasets()

## Training a Model

In [ ]:
from modeling import train_classifier

In [ ]:
# making the species classifier
train_classifier(
  model_name="../classifiers/subspecies_effnet_classifier",
  input_shape=INPUT_SHAPE,
  classes_to_classify=200,
  configuration=CONF,
  model=efficient_net, 
  train_dataset=sub_species_cached_train,
  validation_dataset=sub_species_cached_val,
)

In [ ]:
# making the species classifier
train_classifier(
  model_name="../classifiers/species_efficient_net_classifier",
  input_shape=INPUT_SHAPE,
  classes_to_classify=70,
  configuration=CONF,
  model=big_model, 
  train_dataset=species_cached_train,
  validation_dataset=species_cached_val,
)

## Predict Stuff

In [ ]:
from modeling import predict
import pickle

In [ ]:
# Load mapping for stacked classifiers
with open("../mapping.pickle", "rb+") as f:
  mapping = pickle.load(f)

In [ ]:
# Make Train Dataset
sub_species_input_pipeline.make_test_dataset(directory="../data/test_images/test_images")
test_cached = sub_species_input_pipeline.get_cached_test_datasets()

In [ ]:
predict(
  species_classifier="../classifiers/species_efficient_net_classifier",
  subspecies_classifier="../classifiers/subspecies_pretrained_efficientnet_classifier",
  dataset=test_cached,
  species_subspecies_dict=mapping
)

In [ ]:
# TF predicts labels from 0-199, however we need 1-200: Therefore we need to adjust the predictions.
adj_df_src = "../data/test_images_sample_1701013215.0550277.csv"
pred_df = pd.read_csv(adj_df_src, index_col=0)
pred_df["label"] += 1
pred_df.to_csv(adj_df_src)